# Data preparation
## Read and show Excel data for one day from Flightradar

In [1]:
import pandas as pd

In [3]:
# XLSX-Datei laden
flightradar = pd.read_excel("250503_flightradar.xlsx")

# Erste 5 Zeilen anzeigen
print(flightradar.head(5))  

  web-scraper-order                              web-scraper-start-url  page  \
0      1746350608-1  https://www.flightradar24.com/data/airports/zr...   NaN   
1      1746350608-2  https://www.flightradar24.com/data/airports/zr...   NaN   
2      1746350608-3  https://www.flightradar24.com/data/airports/zr...   NaN   
3      1746350608-4  https://www.flightradar24.com/data/airports/zr...   NaN   
4      1746350608-5  https://www.flightradar24.com/data/airports/zr...   NaN   

   page2  page3  page4  page5  page6  page7      TIME  FLIGHT  \
0    NaN    NaN    NaN    NaN    NaN    NaN  05:45:00   WK186   
1    NaN    NaN    NaN    NaN    NaN    NaN  05:45:00  W22160   
2    NaN    NaN    NaN    NaN    NaN    NaN  05:45:00   CS600   
3    NaN    NaN    NaN    NaN    NaN    NaN  06:00:00   WK130   
4    NaN    NaN    NaN    NaN    NaN    NaN  06:00:00   WK398   

       DESTINATION                                 AIRLINE       AIRCRAFT  \
0   Antalya (AYT)-  Edelweiss Air (Help Alliance Li

## Show data types and shape of the DataFrame

In [4]:
print(flightradar.dtypes)
print(flightradar.shape)  

web-scraper-order         object
web-scraper-start-url     object
page                     float64
page2                    float64
page3                    float64
page4                    float64
page5                    float64
page6                    float64
page7                    float64
TIME                      object
FLIGHT                    object
DESTINATION               object
AIRLINE                   object
AIRCRAFT                  object
STATUS                    object
dtype: object
(362, 15)


## The Date is missing in the DataFrame. Create a new field DATETIME
### DATETIME is the planned departure time for that day

In [5]:
# Datum als Variable definieren
date_str = "2025-05-03"

In [6]:
# TIME als String sicherstellen (falls es z. B. float ist)
flightradar["TIME"] = flightradar["TIME"].astype(str)

# DATETIME-Feld erzeugen: 'date_str' + Zeit aus TIME-Feld
flightradar["DATETIME"] = date_str + " " + flightradar["TIME"]

# Kontrolle
print(flightradar[["TIME", "DATETIME"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

        TIME             DATETIME
0   05:45:00  2025-05-03 05:45:00
1   05:45:00  2025-05-03 05:45:00
2   05:45:00  2025-05-03 05:45:00
3   06:00:00  2025-05-03 06:00:00
4   06:00:00  2025-05-03 06:00:00
5   06:10:00  2025-05-03 06:10:00
6   06:10:00  2025-05-03 06:10:00
7   06:20:00  2025-05-03 06:20:00
8   06:20:00  2025-05-03 06:20:00
9   06:25:00  2025-05-03 06:25:00
10  06:25:00  2025-05-03 06:25:00
11  06:25:00  2025-05-03 06:25:00
12  06:30:00  2025-05-03 06:30:00
13  06:30:00  2025-05-03 06:30:00
14  06:30:00  2025-05-03 06:30:00
15  06:30:00  2025-05-03 06:30:00
16  06:30:00  2025-05-03 06:30:00
17  06:35:00  2025-05-03 06:35:00
18  06:40:00  2025-05-03 06:40:00
19  06:45:00  2025-05-03 06:45:00
20  06:50:00  2025-05-03 06:50:00
21  06:50:00  2025-05-03 06:50:00
22  06:50:00  2025-05-03 06:50:00
23  06:55:00  2025-05-03 06:55:00
24  06:55:00  2025-05-03 06:55:00
25  06:55:00  2025-05-03 06:55:00
26  07:00:00  2025-05-03 07:00:00
27  07:00:00  2025-05-03 07:00:00
28  07:00:00  

## Create two new fields: DESTINATION_CLEAN and IATA_CODE using regex

In [7]:
# DESTINATION_CLEAN: Alles vor der Klammer
flightradar["DESTINATION_CLEAN"] = flightradar["DESTINATION"].str.extract(r"^(.*?)\s*\(")

# IATA-CODE: Inhalt in der Klammer
flightradar["IATA_CODE"] = flightradar["DESTINATION"].str.extract(r"\((\w{3})\)")

# Kontrolle
print(flightradar[["DESTINATION", "DESTINATION_CLEAN", "IATA_CODE"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

                 DESTINATION  DESTINATION_CLEAN IATA_CODE
0             Antalya (AYT)-            Antalya       AYT
1            Pristina (PRN)-           Pristina       PRN
2            Pristina (PRN)-           Pristina       PRN
3            Hurghada (HRG)-           Hurghada       HRG
4             Catania (CTA)-            Catania       CTA
5            Tenerife (TFS)-           Tenerife       TFS
6             Funchal (FNC)-            Funchal       FNC
7               Porto (OPO)-              Porto       OPO
8              Lisbon (LIS)-             Lisbon       LIS
9             Larnaca (LCA)-            Larnaca       LCA
10           Cagliari (CAG)-           Cagliari       CAG
11           Pristina (PRN)-           Pristina       PRN
12             Tromso (TOS)-             Tromso       TOS
13           Hurghada (HRG)-           Hurghada       HRG
14            Larnaca (LCA)-            Larnaca       LCA
15           Hurghada (HRG)-           Hurghada       HRG
16            

## Create three new fields: CANCELED, DEPART_TIME and DEPART_DATETIME using regex

In [8]:
# 1. CANCELED-Spalte (Boolean)
flightradar["CANCELED"] = flightradar["STATUS"].str.contains("Canceled", na=False)

# 2. DEPART_TIME-Spalte (nur wenn "Departed HH:MM" enthalten ist)
flightradar["DEPART_TIME"] = flightradar["STATUS"].str.extract(r"Departed\s*(\d{2}:\d{2})")[0]

# 3. DEPART_DATETIME-Spalte: date_str + DEPART_TIME (fallback "00:00") + ":00"
flightradar["DEPART_DATETIME"] = (
    date_str
    + " "
    + flightradar["DEPART_TIME"].fillna("00:00") # falls keine Zeit da ist, setze "00:00"
    + ":00" # Sekunden anhängen
)

# Kontrolle
print(flightradar[["STATUS","CANCELED","DEPART_TIME","DEPART_DATETIME"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

            STATUS  CANCELED DEPART_TIME      DEPART_DATETIME
0   Departed 06:00     False       06:00  2025-05-03 06:00:00
1          Unknown     False         NaN  2025-05-03 00:00:00
2   Departed 06:02     False       06:02  2025-05-03 06:02:00
3   Departed 06:05     False       06:05  2025-05-03 06:05:00
4   Departed 06:07     False       06:07  2025-05-03 06:07:00
5   Departed 06:31     False       06:31  2025-05-03 06:31:00
6   Departed 06:28     False       06:28  2025-05-03 06:28:00
7   Departed 06:36     False       06:36  2025-05-03 06:36:00
8   Departed 06:44     False       06:44  2025-05-03 06:44:00
9   Departed 06:42     False       06:42  2025-05-03 06:42:00
10  Departed 07:41     False       07:41  2025-05-03 07:41:00
11  Departed 06:37     False       06:37  2025-05-03 06:37:00
12  Departed 06:34     False       06:34  2025-05-03 06:34:00
13         Unknown     False         NaN  2025-05-03 00:00:00
14         Unknown     False         NaN  2025-05-03 00:00:00
15  Depa

## Creating the AIRLINE_CLEAN field and removing unimportant parts

In [9]:
flightradar["AIRLINE_CLEAN"] = (
    flightradar["AIRLINE"]
      .str.extract(r'^(.+?)(?=\s*(?:\(|-))')[0]  # Gruppe 1: alles bis vor "(" oder "-"
      .str.strip()                               # führende/trailing Leerzeichen entfernen
)

# Kontrolle
print(flightradar[["AIRLINE","AIRLINE_CLEAN"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

                                   AIRLINE     AIRLINE_CLEAN
0   Edelweiss Air (Help Alliance Livery) -     Edelweiss Air
1                             Flexflight -        Flexflight
2                         Chair Airlines -    Chair Airlines
3                          Edelweiss Air -     Edelweiss Air
4                                  Swiss -             Swiss
5                          Edelweiss Air -     Edelweiss Air
6                          Edelweiss Air -     Edelweiss Air
7                                  Swiss -             Swiss
8                       TAP Air Portugal -  TAP Air Portugal
9                          Edelweiss Air -     Edelweiss Air
10                         Edelweiss Air -     Edelweiss Air
11                         Edelweiss Air -     Edelweiss Air
12                         Edelweiss Air -     Edelweiss Air
13                            Flexflight -        Flexflight
14                            Flexflight -        Flexflight
15                      

## Creating the AIRCRAFT_SHORT field

In [10]:
# Create the AIRCRAFT_SHORT field by removing text inside parentheses
flightradar["AIRCRAFT_SHORT"] = (
    flightradar["AIRCRAFT"]
      .str.replace(r"\s*\(.*?\)", "", regex=True)  # non‑greedy, regex=True
      .str.strip()
)

# Check the result
print(flightradar[["AIRCRAFT", "AIRCRAFT_SHORT"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

         AIRCRAFT AIRCRAFT_SHORT
0   A320 (HB-JLT)           A320
1          320 ()            320
2   A320 (HB-JOS)           A320
3   A320 (HB-JLS)           A320
4   A320 (HB-IJI)           A320
5   A320 (HB-IJU)           A320
6   A320 (HB-IHX)           A320
7   BCS3 (HB-JCT)           BCS3
8   A321 (CS-TJE)           A321
9   A320 (HB-IJV)           A320
10  A320 (HB-IJW)           A320
11  A359 (HB-IHF)           A359
12  A320 (HB-JLR)           A320
13         320 ()            320
14         319 ()            319
15  A320 (HB-JOK)           A320
16  A319 (HB-JOJ)           A319
17  E290 (HB-AZC)           E290
18  A320 (HB-IHZ)           A320
19  A21N (HB-JPC)           A21N
20  E290 (HB-AZG)           E290
21  A21N (HB-JPD)           A21N
22  BCS1 (HB-JBC)           BCS1
23  E195 (HB-JVJ)           E195
24  BCS3 (YL-AAW)           BCS3
25  E295 (HB-AZL)           E295
26  B738 (PH-BGA)           B738
27  E190 (HB-JVO)           E190
28  BCS3 (HB-JCF)           BCS3
29  BCS3 (

## Create a new field DELAY and DELAY_MINUTES and calculate it from DATETIME and DEPART_DATETIME

In [11]:
# 1) Stelle sicher, dass beide Spalten als datetime64 vorliegen
flightradar["DATETIME"] = pd.to_datetime(flightradar["DATETIME"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
flightradar["DEPART_DATETIME"] = pd.to_datetime(flightradar["DEPART_DATETIME"], format="%Y-%m-%d %H:%M:%S", errors="coerce")

# 2) DELAY als Differenz berechnen
flightradar["DELAY"] = flightradar["DEPART_DATETIME"] - flightradar["DATETIME"]

# 3a) Variante A: Floor‑Division auf total_seconds, dann in int
flightradar["DELAY_MINUTES"] = (
    flightradar["DELAY"].dt.total_seconds() // 60
).astype(int)

# Kontrolle
print(flightradar[["DATETIME","DEPART_DATETIME","DELAY","DELAY_MINUTES"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

              DATETIME     DEPART_DATETIME             DELAY  DELAY_MINUTES
0  2025-05-03 05:45:00 2025-05-03 06:00:00   0 days 00:15:00             15
1  2025-05-03 05:45:00 2025-05-03 00:00:00 -1 days +18:15:00           -345
2  2025-05-03 05:45:00 2025-05-03 06:02:00   0 days 00:17:00             17
3  2025-05-03 06:00:00 2025-05-03 06:05:00   0 days 00:05:00              5
4  2025-05-03 06:00:00 2025-05-03 06:07:00   0 days 00:07:00              7
5  2025-05-03 06:10:00 2025-05-03 06:31:00   0 days 00:21:00             21
6  2025-05-03 06:10:00 2025-05-03 06:28:00   0 days 00:18:00             18
7  2025-05-03 06:20:00 2025-05-03 06:36:00   0 days 00:16:00             16
8  2025-05-03 06:20:00 2025-05-03 06:44:00   0 days 00:24:00             24
9  2025-05-03 06:25:00 2025-05-03 06:42:00   0 days 00:17:00             17
10 2025-05-03 06:25:00 2025-05-03 07:41:00   0 days 01:16:00             76
11 2025-05-03 06:25:00 2025-05-03 06:37:00   0 days 00:12:00             12
12 2025-05-0

## Showing full DataFrame

In [12]:
flightradar.head(3)

,web-scraper-order,web-scraper-start-url,page,page2,page3,page4,page5,page6,page7,TIME,...,DATETIME,DESTINATION_CLEAN,IATA_CODE,CANCELED,DEPART_TIME,DEPART_DATETIME,AIRLINE_CLEAN,AIRCRAFT_SHORT,DELAY,DELAY_MINUTES
0,1746350608-1,https://www.flightradar24.com/data/airports/zr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05:45:00,...,2025-05-03 05:45:00,Antalya,AYT,False,06:00,2025-05-03 06:00:00,Edelweiss Air,A320,0 days 00:15:00,15
1,1746350608-2,https://www.flightradar24.com/data/airports/zr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05:45:00,...,2025-05-03 05:45:00,Pristina,PRN,False,NaN,2025-05-03 00:00:00,Flexflight,320,-1 days +18:15:00,-345
2,1746350608-3,https://www.flightradar24.com/data/airports/zr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05:45:00,...,2025-05-03 05:45:00,Pristina,PRN,False,06:02,2025-05-03 06:02:00,Chair Airlines,A320,0 days 00:17:00,17


## Creating a New DataFrame with Selected Columns

In [13]:
# Create a new DataFrame with selected columns
flightradar_clean = flightradar[[
    "DESTINATION_CLEAN", 
    "IATA_CODE", 
    "AIRLINE_CLEAN", 
    "AIRCRAFT_SHORT", 
    "CANCELED", 
    "DATETIME",
    "DEPART_TIME", 
    "DEPART_DATETIME", 
    "DELAY_MINUTES"
]].copy()

# Check the result
print(flightradar_clean.head(50))
print("\n")
print(flightradar_clean.dtypes)
print(flightradar_clean.shape)

    DESTINATION_CLEAN IATA_CODE     AIRLINE_CLEAN AIRCRAFT_SHORT  CANCELED  \
0             Antalya       AYT     Edelweiss Air           A320     False   
1            Pristina       PRN        Flexflight            320     False   
2            Pristina       PRN    Chair Airlines           A320     False   
3            Hurghada       HRG     Edelweiss Air           A320     False   
4             Catania       CTA             Swiss           A320     False   
5            Tenerife       TFS     Edelweiss Air           A320     False   
6             Funchal       FNC     Edelweiss Air           A320     False   
7               Porto       OPO             Swiss           BCS3     False   
8              Lisbon       LIS  TAP Air Portugal           A321     False   
9             Larnaca       LCA     Edelweiss Air           A320     False   
10           Cagliari       CAG     Edelweiss Air           A320     False   
11           Pristina       PRN     Edelweiss Air           A359

## Count missing values in columns

In [14]:
# Count missing values per column
print(flightradar_clean.isnull().sum())

# Total number of missing values in the DataFrame
print("Total missing values:", flightradar_clean.isnull().sum().sum())

DESTINATION_CLEAN    0
IATA_CODE            0
AIRLINE_CLEAN        5
AIRCRAFT_SHORT       0
CANCELED             0
DATETIME             0
DEPART_TIME          8
DEPART_DATETIME      0
DELAY_MINUTES        0
dtype: int64
Total missing values: 13


## Show rows with missing values in the DataFrame

In [15]:
# Filter rows with any missing values
missing_rows = flightradar_clean[flightradar_clean.isnull().any(axis=1)]

# Display rows with missing values
print(missing_rows)


     DESTINATION_CLEAN IATA_CODE     AIRLINE_CLEAN AIRCRAFT_SHORT  CANCELED  \
1             Pristina       PRN        Flexflight            320     False   
13            Hurghada       HRG        Flexflight            320     False   
14             Larnaca       LCA        Flexflight            319     False   
52              Madrid       MAD            Iberia           A20N     False   
82             Leipzig       LEJ               NaN           PC12     False   
103               Sylt       GWT               NaN           C25M     False   
107           Florence       FLR               NaN           C525     False   
132  Palma de Mallorca       PMI        Flexflight            320     False   
134              Paris       LBG               NaN           CL60     False   
202           Istanbul       ISL               NaN           GLF6     False   
211           Istanbul       IST  Turkish Airlines            33X     False   
305          Frankfurt       FRA             Swiss  

## Show the DataFrame without missing values

In [16]:
# Drop rows with any missing values (NaN)
flightradar_clean = flightradar_clean.dropna(axis=0)

# Display the DataFrame after dropping rows with missing values
print(flightradar_clean)


    DESTINATION_CLEAN IATA_CODE     AIRLINE_CLEAN AIRCRAFT_SHORT  CANCELED  \
0             Antalya       AYT     Edelweiss Air           A320     False   
2            Pristina       PRN    Chair Airlines           A320     False   
3            Hurghada       HRG     Edelweiss Air           A320     False   
4             Catania       CTA             Swiss           A320     False   
5            Tenerife       TFS     Edelweiss Air           A320     False   
..                ...       ...               ...            ...       ...   
357         Hong Kong       HKG             Swiss           B77W     False   
358         Singapore       SIN             Swiss           B77W     False   
359      Johannesburg       JNB             Swiss           A343     False   
360             Milan       MXP  Helvetic Airways           E195     False   
361           Tbilisi       TBS     Edelweiss Air           A320     False   

               DATETIME DEPART_TIME     DEPART_DATETIME  DELAY_

## Check for duplicate rows in the DataFrame and display them if any exist

In [17]:
# Duplikate prüfen
duplicates = flightradar_clean[flightradar_clean.duplicated()]

# Duplikate anzeigen (falls vorhanden)
print(duplicates)

# Anzahl der Duplikate
print(f"Number of duplicate rows: {duplicates.shape[0]}")

Empty DataFrame
Columns: [DESTINATION_CLEAN, IATA_CODE, AIRLINE_CLEAN, AIRCRAFT_SHORT, CANCELED, DATETIME, DEPART_TIME, DEPART_DATETIME, DELAY_MINUTES]
Index: []
Number of duplicate rows: 0


## Save the cleaned DataFrame to a CSV file for further use or analysis

In [19]:
# Speichern des DataFrames als CSV-Datei
flightradar_clean.to_csv("250503_flightradar_prepared.csv", index=False)
